In [ ]:
pip install tf_slim

In [ ]:
pip install mat4py

In [ ]:
pip install lvis

In [ ]:
from mat4py import loadmat

In [ ]:
import os
from shutil import copyfile
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import cv2
import io

In [ ]:
!conda install -y gdown
!gdown https://drive.google.com/uc?id=0B6eKvaijfFUDQUUwd21EckhUbWs

In [ ]:
!unzip -q WIDER_train.zip
!rm WIDER_train.zip

In [ ]:
train_images_path_dict={}
for i in os.listdir('/kaggle/working/WIDER_train/images'):
    for j in os.listdir('/kaggle/working/WIDER_train/images/'+i):
        
        train_images_path_dict[j]='/kaggle/working/WIDER_train/images/'+i+'/'+j
print("Found train images : ",len(train_images_path_dict.keys()))

val_images_path_dict={}
for i in os.listdir('/kaggle/input/wider-face-a-face-detection-benchmark/WIDER_val/WIDER_val/images'):
    for j in os.listdir('/kaggle/input/wider-face-a-face-detection-benchmark/WIDER_val/WIDER_val/images/'+i):
        
        val_images_path_dict[j]='/kaggle/input/wider-face-a-face-detection-benchmark/WIDER_val/WIDER_val/images/'+i+'/'+j
print("Found validation images : ",len(val_images_path_dict.keys()))

In [ ]:
def GetFileAndBbox(filepath,full_path_dict):
    
    data=loadmat(filepath)
    images=[]
    for i in data['file_list']:
        for j in i:
            for k in j:
                for l in k:
                    images.append(full_path_dict[l+'.jpg'])
    print("Found images : ",len(images))
    
    bbox=[]
    for i in data['face_bbx_list']:
        for j in i:
            for k in j:
                for l in k:
                    if isinstance(l[0],int):
                        bbox.append([l])
                    else:
                        bbox.append(l)
    print("Found bbox : ",len(bbox))
    
    return images,bbox

In [ ]:
train_images,train_bbox=GetFileAndBbox('/kaggle/input/wider-face-a-face-detection-benchmark/wider_face_split/wider_face_split/wider_face_train.mat',train_images_path_dict)
val_images,val_bbox=GetFileAndBbox('/kaggle/input/wider-face-a-face-detection-benchmark/wider_face_split/wider_face_split/wider_face_val.mat',val_images_path_dict)

In [ ]:
x,y,w,h=train_bbox[0][0]
img=cv2.cvtColor(cv2.imread(train_images[0]),cv2.COLOR_BGR2RGB)
for i in train_bbox[0]:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),5)
    
plt.imshow(img)

In [ ]:
!wget -O protobuf.zip https://github.com/google/protobuf/releases/download/v3.13.0/protoc-3.13.0-linux-x86_64.zip -q
!unzip -o protobuf.zip
!rm protobuf.zip

In [ ]:
os.environ['PYTHONPATH']=os.environ['PYTHONPATH']+':/kaggle/working:/kaggle/models/research:/kaggle/models/research/slim'

In [ ]:
os.environ['PYTHONPATH'] += ":/kaggle/models"

In [ ]:
os.environ['PYTHONPATH']

In [ ]:
%cd /kaggle
!git clone https://github.com/tensorflow/models.git
!rm -fr models/.git

In [ ]:
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!mkdir /kaggle/working/coco
%cd /kaggle/working/coco
!git clone https://github.com/cocodataset/cocoapi.git
%cd cocoapi/PythonAPI
!make
!cp -r pycocotools /kaggle/models/research/

In [ ]:
%cd /kaggle/models/research
!cp object_detection/packages/tf2/setup.py .

In [ ]:
%cd /kaggle/models/research
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
!mkdir /kaggle/workspace
!mkdir /kaggle/workspace/trainingdemo
%cd /kaggle/workspace/trainingdemo
!mkdir annotations exported-models images models pre-trained-models 
!mkdir images/train images/val images/test
!touch /kaggle/workspace/trainingdemo/README.md
!ls /kaggle/workspace/trainingdemo

In [ ]:
!apt-get install tree

In [ ]:
!tree /kaggle/workspace

In [ ]:
%cd /kaggle/models/research

In [ ]:
from object_detection.utils.dataset_util import bytes_list_feature
from object_detection.utils.dataset_util import float_list_feature
from object_detection.utils.dataset_util import int64_list_feature
from object_detection.utils.dataset_util import int64_feature
from object_detection.utils.dataset_util import bytes_feature
from PIL import Image

In [ ]:
label='face'

In [ ]:
class Annotation:
    
    def __init__(self):
        self.xMins=[]
        self.xMaxs=[]
        self.yMins=[]
        self.yMaxs=[]
        self.textLabels=[]
        self.classes=[]
        self.difficult=[]
        
        self.image=None
        self.Width=None
        self.Height=None
        self.encoding=None
        self.filename=None
        
    def build(self):
        
        w=int64_feature(self.Width)
        h=int64_feature(self.Height)
        filename=bytes_feature(self.filename.encode('utf8'))
        encoding=bytes_feature(self.encoding.encode('utf8'))
        image=bytes_feature(self.image)
        xMins=float_list_feature(self.xMins)
        xMaxs=float_list_feature(self.xMaxs)
        yMins=float_list_feature(self.yMins)
        yMaxs=float_list_feature(self.yMaxs)
        textLabels=bytes_list_feature(self.textLabels)
        classes=int64_list_feature(self.classes)
        difficult=int64_list_feature(self.difficult)
        
        data={
            
            "image/height":h,
            "image/width":w,
            "image/filename":filename,
            "image/source_id":filename,
            "image/encoded":image,
            "image/format":encoding,
            "image/object/bbox/xmin":xMins,
            "image/object/bbox/xmax":xMaxs,
            "image/object/bbox/ymax":yMaxs,
            "image/object/bbox/ymin":yMins,
            "image/object/class/text":textLabels,
            "image/object/class/label":classes,
            "image/object/difficult":difficult
        }
        
        return data

In [ ]:
with open('/kaggle/workspace/trainingdemo/annotations/labelmap.pbtxt','w') as f:
    label=("item{\n"
               "\tid:"+str(1)+"\n"
               "\tname: 'face'"+"\n"
               "}\n")
    f.write(label)
    
    f.close()

In [ ]:
!cat /kaggle/workspace/trainingdemo/annotations/labelmap.pbtxt

In [ ]:
!touch '/kaggle/workspace/trainingdemo/annotations/train.record' '/kaggle/workspace/trainingdemo/annotations/val.record' '/kaggle/workspace/trainingdemo/annotations/test.record'
!ls /kaggle/workspace/trainingdemo/annotations

In [ ]:

datasets=[
    ("train",train_images,train_bbox,'/kaggle/workspace/trainingdemo/annotations/train.record'),
    ("validation",val_images,val_bbox,'/kaggle/workspace/trainingdemo/annotations/val.record')
]

for datasetType,keys,rects_list,recordPath in datasets:
    print("Processing {}...".format(datasetType))
    writer=tf.io.TFRecordWriter(recordPath)
    
    total=0
    
    for k,rects in dict(zip(keys,rects_list)).items():
        encoded=tf.io.gfile.GFile(k,'rb').read()
        #encoded=bytes(encoded)
        
        pilimg=Image.open(k)
        (width,height)=pilimg.size[:2]
        
        filename=k.split('/')[-1]
        encoding=filename.split(".")[-1]
        
        tfannote=Annotation()
        tfannote.image=encoded
        tfannote.encoding=encoding
        tfannote.filename=filename
        tfannote.Width=width
        tfannote.Height=height
        
        for x,y,w,h in rects: 
        
            (label,startX,startY,endX,endY)=('face',x,y,x+w,y+h)

            xMin=startX/width
            xMax=endX/width
            yMin=startY/height
            yMax=endY/height

            tfannote.xMins.append(xMin)
            tfannote.xMaxs.append(xMax)
            tfannote.yMins.append(yMin)
            tfannote.yMaxs.append(yMax)
            tfannote.textLabels.append(label.encode('utf8'))
            tfannote.classes.append(1)
            tfannote.difficult.append(0)
            
            total+=1
            
    
        features=tf.train.Features(feature=tfannote.build())
        example=tf.train.Example(features=features)
    
        writer.write(example.SerializeToString())

    writer.close()
    print("{} examples saved for as {} record".format(total,datasetType))

In [ ]:
!ls -lh /kaggle/workspace/trainingdemo/annotations/

In [ ]:
%cd '/kaggle/workspace/trainingdemo/pre-trained-models/'
!wget -O ssdMobileNetmodel.tar.gz download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8.tar.gz -q
!tar -zxvf ssdMobileNetmodel.tar.gz
!rm ssdMobileNetmodel.tar.gz

In [ ]:
!tree '/kaggle/workspace/trainingdemo/pre-trained-models'

In [ ]:
!cp ssd_mobilenet_v1_fpn_640x640_coco17_tpu-8/pipeline.config /kaggle/working/pipeline.config

In [ ]:
##Download and tweak the config file as per your dataset and hardware needs

In [ ]:
!cp /kaggle/input/ssdmobilenetconfig/pipeline.config /kaggle/workspace/trainingdemo/models/pipeline.config

In [ ]:
!cp /kaggle/models/research/object_detection/model_main_tf2.py /kaggle/workspace/trainingdemo/models

In [ ]:
#!rm -r /kaggle/workspace/trainingdemo/models/train

In [ ]:
%cd /kaggle/models/research

In [ ]:
!python  /kaggle/workspace/trainingdemo/models/model_main_tf2.py --model_dir=/kaggle/workspace/trainingdemo/models --pipeline_config_path=/kaggle/workspace/trainingdemo/models/pipeline.config

In [ ]:
#!python  /kaggle/workspace/trainingdemo/models/model_main_tf2.py --model_dir=/kaggle/workspace/trainingdemo/models --pipeline_config_path=/kaggle/workspace/trainingdemo/models/pipeline.config --checkpoint_dir=--model_dir=/kaggle/workspace/trainingdemo/models

In [ ]:
!pwd

In [ ]:
!cp object_detection/exporter_main_v2.py /kaggle/workspace/trainingdemo/

In [ ]:
%cd /kaggle/workspace/trainingdemo/

In [ ]:
!tree /kaggle/workspace/trainingdemo/

In [ ]:
!python ./exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/pipeline.config --trained_checkpoint_dir ./models/ --output_directory ./exported-models/my_model

In [ ]:
!tree /kaggle/workspace/trainingdemo/exported-models/

In [ ]:
!cp -r /kaggle/workspace/trainingdemo/exported-models/my_model /kaggle/working/SSDResnetDetectionModel

In [ ]:
!rm -r /kaggle/working/bin
!rm -r /kaggle/working/include
!rm -r /kaggle/working/coco
!rm -r /kaggle/working/WIDER_train
!rm /kaggle/working/pipeline.config
!rm /kaggle/working/readme.txt